In [ ]:
嘿嘿，勉强做出来了，只是代码不太优雅，哈哈哈

In [1]:
import numpy as np

import pandas as pd

import matplotlib.pyplot as plt

# Ex4：删除同样的行
现有两张表，请在df1中剔除在df2中出现过的行。

In [17]:
df1 = pd.DataFrame({
    "A": [3,2,2,3,1,3],
    "B": [2,1,1,3,6,2],
    "C": [1,2,2,7,7,1],
    "D": [5,6,6,1,2,5],
})
df2 = pd.DataFrame({
    "A": [2,3,1],
    "B": [1,9,6],
    "C": [2,7,7],
    "D": [6,1,2],
})
df1

,A,B,C,D
0,3,2,1,5
1,2,1,2,6
2,2,1,2,6
3,3,3,7,1
4,1,6,7,2
5,3,2,1,5


In [18]:
df2

,A,B,C,D
0,2,1,2,6
1,3,9,7,1
2,1,6,7,2


In [19]:
#结果应当如下：

pd.DataFrame({
    "A": [3,3,3],
    "B": [2,3,2],
    "C": [1,7,1],
    "D": [5,1,5],
})


,A,B,C,D
0,3,2,1,5
1,3,3,7,1
2,3,2,1,5


In [160]:
s1 = df1.agg(tuple, axis=1)
s2 = df2.agg(tuple, axis=1)

df1[~s1.isin(s2)]#isin：它独立于index。也就是说，它只检查值。只需要将每个数据帧的列转换为一系列元组来创建掩码

,A,B,C,D
0,3,2,1,5
3,3,3,7,1
5,3,2,1,5


In [157]:
df1 = pd.DataFrame({
    "A": [3,2,2,3,1,3],
    "B": [2,1,1,3,6,2],
    "C": [1,2,2,7,7,1],
    "D": [5,6,6,1,2,5],
}, dtype='float')

df2 = pd.DataFrame({
    "A": [2,3,1],
    "B": [1,9,6],
    "C": [2,7,7],
    "D": [6,1,2],
}, dtype='float')

for i in df1.index:
    #print(df1.values[i])
    for j in df2.index:
        if (df1.values[i] == df2.values[j]).all():
            df1.values[i]= np.NaN
#             print(df1)
df1.dropna()  

,A,B,C,D
0,3.0,2.0,1.0,5.0
3,3.0,3.0,7.0,1.0
5,3.0,2.0,1.0,5.0


# Ex5：统计每个学区的开课数量
某个城市共有4个学区，每个学区有若干学校，学校之间名字互不相同。每一条记录为该学校开设的课程，一个学校可能有多条记录，每一条记录内部的课程不会重复，但同一学校不同记录之间的课程可能重复。

In [2]:
df = pd.read_csv('data/supplement/ex5/school_course.csv')

df.head()

,Area,School,Course
0,area_1,school_99,course_90
1,area_2,school_32,course_20
2,area_3,school_64,course_38
3,area_1,school_231,course_9 course_40 course_100
4,area_3,school_147,course_57 course_77 course_28


课程的种类共有100门，编号为”school_1”到”school_100”。现要统计每个学区各项课程的开设学校数量，结果如下格式：

In [66]:
res = pd.DataFrame(
    0, index=["course_%d"%(i+1) for i in range(100)],
    columns=["area_%d"%(i+1) for i in range(4)]
)


res.head() # 若area_1共有20所学校开设了course_1，则第一个单元格为20

,area_1,area_2,area_3,area_4
course_1,0,0,0,0
course_2,0,0,0,0
course_3,0,0,0,0
course_4,0,0,0,0
course_5,0,0,0,0


##### 先分解DataFrame中具有多个值的单元格，然后删除重复值，然后筛选出area_1的数据，根据course进行统计，然后转成dataframe格式，其他地区的数据进行类似的处理，然后拼接起来

In [62]:
# 分解DataFrame中具有多个值的单元格
def expand(df, col, sep=' '):
    r = df[col].str.split(sep)
    d = {c: df[c].values.repeat(r.str.len(), axis=0) for c in df.columns}
    d[col] = [i for sub in r for i in sub]
    return pd.DataFrame(d)

df_expand = expand(df, 'Course')

# 删除重复值
df_expand = df_expand.drop_duplicates()

res1 = df_expand[df_expand['Area'] == 'area_1'].groupby(["Course"]).size().to_frame(name='area_1')
res2 = df_expand[df_expand['Area'] == 'area_2'].groupby(["Course"]).size().to_frame(name='area_2')
res3 = df_expand[df_expand['Area'] == 'area_3'].groupby(["Course"]).size().to_frame(name='area_3')
res4 = df_expand[df_expand['Area'] == 'area_4'].groupby(["Course"]).size().to_frame(name='area_4')

res0 = res1.join(res2, how='left').join(res3, how='left')
res = res0.join(res4, how='left')  # area_4的数据因为存在NaN值，所以变成了浮点型

# 把nan替换成0，把float转int
res4 = res['area_4'].fillna(0).astype(int).to_frame()

res = res0.join(res4, how='left')

res = res.reset_index() # 重新生成一个默认索引

# 对Course进行排序
res.Course = res.Course.astype('category')
f = list(map(lambda i: "course_%s"%(i+1), range(100)))
res.Course = res.Course.cat.reorder_categories(f,ordered=True)
res = res.sort_values('Course')

# 将Course设置为索引
res.set_index('Course')

,area_1,area_2,area_3,area_4
Course,,,,
course_1,7,13,16,4
course_2,3,13,15,4
course_3,5,16,13,7
course_4,5,14,13,9
course_5,3,7,13,6
...,...,...,...,...
course_96,5,9,12,9
course_97,3,8,11,2
course_98,4,10,14,8


In [46]:
# 分解DataFrame中具有多个值的单元格
def expand(df, col, sep=' '):
    r = df[col].str.split(sep)
    d = {c: df[c].values.repeat(r.str.len(), axis=0) for c in df.columns}
    d[col] = [i for sub in r for i in sub]
    return pd.DataFrame(d)

df_expand = expand(df, 'Course')

In [47]:
df_expand

,Area,School,Course
0,area_1,school_99,course_90
1,area_2,school_32,course_20
2,area_3,school_64,course_38
3,area_1,school_231,course_9
4,area_1,school_231,course_40
...,...,...,...
3848,area_3,school_349,course_27
3849,area_3,school_349,course_18
3850,area_2,school_62,course_31
3851,area_2,school_62,course_22


In [48]:
# 删除重复值
df_expand = df_expand.drop_duplicates()

In [49]:
df_expand

,Area,School,Course
0,area_1,school_99,course_90
1,area_2,school_32,course_20
2,area_3,school_64,course_38
3,area_1,school_231,course_9
4,area_1,school_231,course_40
...,...,...,...
3846,area_2,school_242,course_51
3847,area_2,school_92,course_38
3850,area_2,school_62,course_31
3851,area_2,school_62,course_22


In [50]:
df1 = df_expand[df_expand['Area'] == 'area_1']
df1_gb = df1.groupby(["Course"]).size()
df1_gb

Course
course_1      7
course_10     7
course_100    4
course_11     8
course_12     3
             ..
course_95     8
course_96     5
course_97     3
course_98     4
course_99     4
Length: 100, dtype: int64

In [51]:
df2 = df_expand[df_expand['Area'] == 'area_2']
df2_gb = df2.groupby(["Course"]).size()
df2_gb

Course
course_1      13
course_10     11
course_100    10
course_11      9
course_12      2
              ..
course_95     17
course_96      9
course_97      8
course_98     10
course_99      9
Length: 100, dtype: int64

In [52]:
df3 = df_expand[df_expand['Area'] == 'area_3']
df3_gb = df3.groupby(["Course"]).size()
df3_gb

Course
course_1      16
course_10     19
course_100     9
course_11     13
course_12     11
              ..
course_95     12
course_96     12
course_97     11
course_98     14
course_99     16
Length: 100, dtype: int64

In [53]:
df4 = df_expand[df_expand['Area'] == 'area_4']
df4_gb = df4.groupby(["Course"]).size()
df4_gb

Course
course_1      4
course_10     9
course_100    7
course_12     7
course_13     6
             ..
course_95     5
course_96     9
course_97     2
course_98     8
course_99     8
Length: 99, dtype: int64

In [54]:
res1 = df1_gb.to_frame(name='area_1')
res2 = df2_gb.to_frame(name='area_2')
res3 = df3_gb.to_frame(name='area_3')
res4 = df4_gb.to_frame(name='area_4')

In [55]:
res0 = res1.join(res2, how='left').join(res3, how='left')
res = res0.join(res4, how='left')
res
# area_4的数据因为存在NaN值，所以变成了浮点型

,area_1,area_2,area_3,area_4
Course,,,,
course_1,7,13,16,4.0
course_10,7,11,19,9.0
course_100,4,10,9,7.0
course_11,8,9,13,NaN
course_12,3,2,11,7.0
...,...,...,...,...
course_95,8,17,12,5.0
course_96,5,9,12,9.0
course_97,3,8,11,2.0


In [56]:
# 把nan替换成0，把float转int
res4 = res['area_4'].fillna(0).astype(int).to_frame()
res4

,area_4
Course,
course_1,4
course_10,9
course_100,7
course_11,0
course_12,7
...,...
course_95,5
course_96,9
course_97,2


In [57]:
res = res0.join(res4, how='left')
res

,area_1,area_2,area_3,area_4
Course,,,,
course_1,7,13,16,4
course_10,7,11,19,9
course_100,4,10,9,7
course_11,8,9,13,0
course_12,3,2,11,7
...,...,...,...,...
course_95,8,17,12,5
course_96,5,9,12,9
course_97,3,8,11,2


In [58]:
f = list(map(lambda i: "course_%s"%(i+1), range(100)))
f

['course_1',
 'course_2',
 'course_3',
 'course_4',
 'course_5',
 'course_6',
 'course_7',
 'course_8',
 'course_9',
 'course_10',
 'course_11',
 'course_12',
 'course_13',
 'course_14',
 'course_15',
 'course_16',
 'course_17',
 'course_18',
 'course_19',
 'course_20',
 'course_21',
 'course_22',
 'course_23',
 'course_24',
 'course_25',
 'course_26',
 'course_27',
 'course_28',
 'course_29',
 'course_30',
 'course_31',
 'course_32',
 'course_33',
 'course_34',
 'course_35',
 'course_36',
 'course_37',
 'course_38',
 'course_39',
 'course_40',
 'course_41',
 'course_42',
 'course_43',
 'course_44',
 'course_45',
 'course_46',
 'course_47',
 'course_48',
 'course_49',
 'course_50',
 'course_51',
 'course_52',
 'course_53',
 'course_54',
 'course_55',
 'course_56',
 'course_57',
 'course_58',
 'course_59',
 'course_60',
 'course_61',
 'course_62',
 'course_63',
 'course_64',
 'course_65',
 'course_66',
 'course_67',
 'course_68',
 'course_69',
 'course_70',
 'course_71',
 'course_72',
 

In [59]:
res = res.reset_index() # 重新生成一个默认索引
res

,Course,area_1,area_2,area_3,area_4
0,course_1,7,13,16,4
1,course_10,7,11,19,9
2,course_100,4,10,9,7
3,course_11,8,9,13,0
4,course_12,3,2,11,7
...,...,...,...,...,...
95,course_95,8,17,12,5
96,course_96,5,9,12,9
97,course_97,3,8,11,2
98,course_98,4,10,14,8


In [60]:
# 对Course进行排序
res.Course = res.Course.astype('category')
res.Course = res.Course.cat.reorder_categories(f,ordered=True)
res = res.sort_values('Course')
res

,Course,area_1,area_2,area_3,area_4
0,course_1,7,13,16,4
12,course_2,3,13,15,4
23,course_3,5,16,13,7
34,course_4,5,14,13,9
45,course_5,3,7,13,6
...,...,...,...,...,...
96,course_96,5,9,12,9
97,course_97,3,8,11,2
98,course_98,4,10,14,8
99,course_99,4,9,16,8


In [61]:
# 将Course设置为索引
res = res.set_index('Course')
res

,area_1,area_2,area_3,area_4
Course,,,,
course_1,7,13,16,4
course_2,3,13,15,4
course_3,5,16,13,7
course_4,5,14,13,9
course_5,3,7,13,6
...,...,...,...,...
course_96,5,9,12,9
course_97,3,8,11,2
course_98,4,10,14,8


# Ex6：捕获非零的行列索引
给定如下的数据框，请返回非零行列组合构成的多级索引。

In [2]:
df = pd.DataFrame(
    [[0,5,0],[2,1,0],[0,0,6],[0,9,0]],
    index=list("ABCD"), columns=list("XYZ"))


df

,X,Y,Z
A,0,5,0
B,2,1,0
C,0,0,6
D,0,9,0


In [52]:
res = pd.Index([
    ('X', 'B'),
    ('Y', 'A'),
    ('Y', 'B'),
    ('Y', 'D'),
    ('Z', 'C')])


res['X','X','X','X', 'Y','Y','Y','Y', 'Z', 'Z', 'Z', 'Z']

MultiIndex([('X', 'B'),
            ('Y', 'A'),
            ('Y', 'B'),
            ('Y', 'D'),
            ('Z', 'C')],
           )

##### 思路：重新构造datafeame，把原来的值作为新的索引，原来的索引作为新的值，然后把0索引的行去除，剩下的元素则构造多级索引

In [61]:
df2 = pd.DataFrame(data = {'col_0': list(np.array(list(df.columns)*len(df.index)).reshape(len(df.index),len(df.columns)).T.reshape(-1)),
                           'col_1': list(df.index) * len(df.columns)},
                  index = list(df.values.T.reshape(-1)))

df3 = df2.drop(labels=None,axis=0, index=0, columns=None, inplace=False)

my_array = [list(df3['col_0']), list(df3['col_1'])]

res = pd.MultiIndex.from_arrays(my_array)
res

MultiIndex([('X', 'B'),
            ('Y', 'A'),
            ('Y', 'B'),
            ('Y', 'D'),
            ('Z', 'C')],
           )

In [37]:
df2 = pd.DataFrame(data = {'col_0': list("X") * 4 + list("Y") * 4 + list("Z") * 4,
                           'col_1': list("ABCD") * 3},
                  index = list(df['X']) + list(df['Y']) + list(df['Z']))

df3 = df2.drop(labels=None,axis=0, index=0, columns=None, inplace=False)

my_array = [list(df3['col_0']), list(df3['col_1'])]

res = pd.MultiIndex.from_arrays(my_array)
res

MultiIndex([('X', 'B'),
            ('Y', 'A'),
            ('Y', 'B'),
            ('Y', 'D'),
            ('Z', 'C')],
           )

In [38]:
list("X") * 4 + list("Y") * 4 + list("Z") * 4

['X', 'X', 'X', 'X', 'Y', 'Y', 'Y', 'Y', 'Z', 'Z', 'Z', 'Z']

In [54]:
list(np.array(list(df.columns)*len(df.index)).reshape(len(df.index),len(df.columns)).T.reshape(-1))

['X', 'X', 'X', 'X', 'Y', 'Y', 'Y', 'Y', 'Z', 'Z', 'Z', 'Z']

In [62]:
list("ABCD") * 3

['A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D']

In [63]:
list(df.index) * len(df.columns)

['A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D']

In [56]:
list(df['X']) + list(df['Y']) + list(df['Z'])

[0, 2, 0, 0, 5, 1, 0, 9, 0, 0, 6, 0]

In [60]:
list(df.values.T.reshape(-1))

[0, 2, 0, 0, 5, 1, 0, 9, 0, 0, 6, 0]